In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gluonts.dataset.pandas import PandasDataset, is_uniform, infer_freq
from gluonts.dataset.split import split, TestData
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_single, plot_next_multi
from uni2ts.eval_util.evaluation import evaluate_model
from gluonts.ev.metrics import MAE, MAPE
from gluonts.evaluation.metrics import quantile_loss

from fts_explore.common.benchmark_func import get_model_data, get_eval_foreasts

In [ ]:
TITLE = "Stage 2 Finetuning: FT vs PT"
XLABEL = "Data available in 2-month patches"
TOTAL_TIME_STEPS = 25
PDT = 7
CTX = 30

In [ ]:
data = pd.read_csv("../data/eth_15_24.csv", delimiter=",")
data["DateUTC"] = pd.to_datetime(data["DateUTC"])
data = data.set_index("DateUTC")

In [ ]:
def get_metrics_v2(
    model_folder: str, data: pd.DataFrame, total_time_steps: int, prediction_lenght: int
):
    metrics = {
        "mean": [],
        "median": [],
        "lower_0025": [],
        "upper_0975": [],
        "mean_wql": [],
    }
    for i in range(1, total_time_steps + 1):
        try:
            model, test_data = get_model_data(
                model_folder=model_folder,
                prediction_lenght=prediction_lenght,
                time_steps=i,
                time_step_size=prediction_lenght,
                max_context_length=CTX,
                target_var="ETH-USD",
                data=data,
                patch_size=16,
                num_samples=200,
                freq="D",
            )

            forecast_samples, target_values = get_eval_foreasts(model, test_data)

            # absolute error of the average forecast of each time step
            mean_error_ts = (
                np.mean(forecast_samples, axis=1).flatten() - target_values.flatten()
            ) / target_values.flatten()
            mean_error_ts = np.abs(mean_error_ts)

            # absolute error of the median forecast of each time step
            median_error_ts = (
                np.quantile(forecast_samples, 0.5, axis=1).flatten()
                - target_values.flatten()
            ) / target_values.flatten()
            median_error_ts = np.abs(median_error_ts)

            # absolute error of the 97.5th percentile forecast of each time step
            upper_error_ts = (
                np.quantile(forecast_samples, 0.975, axis=1).flatten()
                - target_values.flatten()
            ) / target_values.flatten()
            upper_error_ts = np.abs(upper_error_ts)

            # absolute error of the 2.5th percentile forecast of each time step
            lower_error_ts = (
                np.quantile(forecast_samples, 0.025, axis=1).flatten()
                - target_values.flatten()
            ) / target_values.flatten()
            lower_error_ts = np.abs(lower_error_ts)

            w_q_loss = [
                np.sum(
                    quantile_loss(
                        target_values.flatten(),
                        np.quantile(forecast_samples, q, axis=1).flatten(),
                        q=q,
                    )
                )
                / np.sum(target_values.flatten())
                for q in np.arange(0.1, 1.0, 0.1)
            ]
            mean_w_q_loss = np.mean(w_q_loss)

            metrics["mean"].append(mean_error_ts)
            metrics["median"].append(median_error_ts)
            metrics["lower_0025"].append(lower_error_ts)
            metrics["upper_0975"].append(upper_error_ts)
            metrics["mean_wql"].append(mean_w_q_loss)

        except Exception as e:
            print(e)

    return metrics

In [ ]:
model_folder = "../outputs/finetune/stage_two_ft_crypto/checkpoints/eth_data_train_"
stage_2_ft = get_metrics_v2(
    model_folder, data, total_time_steps=TOTAL_TIME_STEPS, prediction_lenght=PDT
)

model_folder = "../outputs/finetune/stage_two_pt_crypto/checkpoints/eth_data_train_"
stage_2_pt = get_metrics_v2(
    model_folder, data, total_time_steps=TOTAL_TIME_STEPS, prediction_lenght=PDT
)

model_folder = "pretrained"
pretrained = get_metrics_v2(
    model_folder, data, total_time_steps=TOTAL_TIME_STEPS, prediction_lenght=PDT
)

In [ ]:
stage_2_ft = pd.DataFrame.from_dict(stage_2_ft)
stage_2_pt = pd.DataFrame.from_dict(stage_2_pt)
pretrained = pd.DataFrame.from_dict(pretrained)

stage_2_ft.to_csv(
    f"../experiment_results/stage_2_finetune/crypto_data/ft_{PDT}_{CTX}_performance.csv",
    index=False,
)
stage_2_pt.to_csv(
    f"../experiment_results/stage_2_finetune/crypto_data/pt_{PDT}_{CTX}_performance.csv",
    index=False,
)
pretrained.to_csv(
    f"../experiment_results/stage_2_finetune/crypto_data/pretrained_{PDT}_{CTX}_performance.csv",
    index=False,
)

In [ ]:
# MAPE of the average forecast
plt.title(TITLE)
plt.ylabel("MAPE of the mean forecast")
plt.xlabel(XLABEL)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in stage_2_ft["mean"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from FT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in stage_2_pt["mean"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from PT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in pretrained["mean"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="Pretrained",
)
plt.legend()
plt.savefig(
    "../experiment_results/stage_2_finetune/crypto_data/stage_2_main_comp_mean.jpeg"
);

In [ ]:
# MAPE of the 97.5th percentile forecast
plt.title(TITLE)
plt.ylabel("MAPE of the 97.5th percentile forecast")
plt.xlabel(XLABEL)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in stage_2_ft["upper_0975"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from FT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in stage_2_pt["upper_0975"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from PT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in pretrained["upper_0975"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="Pretrained",
)
plt.legend()
plt.savefig(
    "../experiment_results/stage_2_finetune/crypto_data/stage_2_main_comp_975.jpeg"
);

In [ ]:
# MAPE of the 2.5th percentile forecast
plt.title(TITLE)
plt.ylabel("MAPE of the 2.5th forecast")
plt.xlabel(XLABEL)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in stage_2_ft["lower_0025"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from FT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in stage_2_pt["lower_0025"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from PT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in pretrained["lower_0025"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="Pretrained",
)
plt.legend()
plt.savefig(
    "../experiment_results/stage_2_finetune/crypto_data/stage_2_main_comp_025.jpeg"
);

In [ ]:
# MAPE of the median forecast
plt.title(TITLE)
plt.ylabel("MAPE of the median forecast")
plt.xlabel(XLABEL)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in stage_2_ft["median"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from FT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in stage_2_pt["median"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from PT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=[np.mean(ts) for ts in pretrained["median"].tolist()],
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="Pretrained",
)
plt.legend()
plt.savefig(
    "../experiment_results/stage_2_finetune/crypto_data/stage_2_main_comp_median.jpeg"
);

In [ ]:
# MAPE of the average forecast
plt.title(TITLE)
plt.ylabel("Mean Weighted Quantile Loss")
plt.xlabel(XLABEL)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=stage_2_ft["mean_wql"].tolist(),
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from FT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=stage_2_pt["mean_wql"].tolist(),
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="from PT",
)
plt.errorbar(
    x=range(1, TOTAL_TIME_STEPS + 1),
    y=pretrained["mean_wql"].tolist(),
    ecolor="red",
    barsabove=True,
    linestyle="dotted",
    marker=".",
    errorevery=2,
    label="Pretrained",
)
plt.legend()
plt.savefig(
    "../experiment_results/stage_2_finetune/crypto_data/stage_2_main_comp_mean_wql.jpeg"
);